In [10]:
import numpy as np
from submit_2 import my_map
# from submit_2 import my_fit
from scipy.linalg import khatri_rao
import time as tm

In [11]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [12]:
def my_fit( X_train, y_train ):
################################
#  Non Editable Region Ending  #
################################
	feat = my_map(X_train)
	model = LogisticRegression(penalty='l2', C=62.0, fit_intercept=True, max_iter=2500)
	# Use this method to train your model using training CRPs
	# X_train has 32 columns containing the challeenge bits
	# y_train contains the responses
	model.fit(feat, y_train)
	w = model.coef_[0]
	b = model.intercept_
	
	# THE RETURNED MODEL SHOULD BE A SINGLE VECTOR AND A BIAS TERM
	# If you do not wish to use a bias term, set it to 0
	return w, b

In [9]:
def my_map(X):
    X = np.hstack((X, np.ones((X.shape[0], 1))))
    X = 1 - 2 * X

    # Compute the cumulative products
    cumprod_X = np.cumprod(X[:, ::-1], axis=1)[:, ::-1]

    # Compute the feature interactions directly
    feat = np.ones((X.shape[0], X.shape[1] * (X.shape[1] - 1) // 2 + 1))

    index = 0
    for i in range(X.shape[1]):
        for j in range(i + 1, X.shape[1]):
            feat[:, index] = cumprod_X[:, i] * cumprod_X[:, j]
            index += 1

    return feat

In [10]:
def my_map__(X):
    X_with_ones = np.column_stack((X, np.ones(X.shape[0])))

    # Compute 1 - 2 * X using broadcasting
    X_transformed = 1 - 2 * X_with_ones

    # Compute cumulative products using vectorized operation
    cumprod_X = np.flip(np.cumprod(np.flip(X_transformed, axis=1), axis=1), axis=1)

    # Compute feature interactions without explicit loops
    num_features = X.shape[1]
    indices = np.triu_indices(num_features, 1)
    feat = np.prod(cumprod_X[:, indices[0]] * cumprod_X[:, indices[1]], axis=2)

    # Append a column of ones as the last feature
    feat = np.column_stack((feat, np.ones(X.shape[0])))

    return feat

In [11]:
# def my_map(X):
# ################################
# #  Non Editable Region Ending  #
# ################################
# 	X = np.hstack((X, np.ones((X.shape[0], 1))))
# 	X = 1 - 2 * X
# 	# print(X.shape)
	
# 	arr = []
# 	for i in range(X.shape[0]):
# 		temp = []
# 		temp.append(X[i][-1])
# 		for j in range(X.shape[1] - 2, -1, -1):
# 			temp.append(X[i][j] * temp[-1])
# 		temp.reverse()
# 		arr.append(np.array(temp))
    
# 	arr = np.array(arr)
# 	# print(arr.shape)
# 	feat = []
# 	for i in range(X.shape[0]):
# 		res = []
# 		for j in range(X.shape[1]):
# 			for k in range(j+1, X.shape[1]):
# 				res.append(arr[i][j] * arr[i][k])
# 		feat.append(np.array(res))

# 	return np.array(feat)

# 	# # Use this method to create features.
# 	# # It is likely that my_fit will internally call my_map to create features for train points

# 	# return feat

In [13]:
Z_trn = np.loadtxt( "dummy/secret_train.dat" )
Z_tst = np.loadtxt( "dummy/secret_test.dat" )

n_trials = 5

bias = 0
d_size = 0
t_train = 0
t_map = 0
acc = 0

In [14]:
my_map( Z_tst[:, :-1] ).shape

(10000, 528)

In [15]:
my_map( Z_tst[:, :-1] ).shape

(10000, 528)

In [16]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic

	d_size += w.shape[0]
	bias += b

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

In [17]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials
bias /= n_trials

In [18]:
print( d_size, t_train, t_map, acc, bias)

528.0 0.6477962165954523 0.025102133199106903 0.9925 [-52.9275518]


In [36]:
print( d_size, t_train, t_map, acc, bias)

528.0 0.3904974831850268 0.025894074782263486 0.8267 [0.]
